In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense


2024-01-11 16:23:22.757949: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
PREFIX = "/Users/moctader/Thesis_code"  # folder with files
PREFIX = "/Users/akusok/wrkdir/Golam"  # folder with files

In [3]:
data_path = f"{PREFIX}/samples.pkl"

In [4]:
# Read Data
df=gpd.GeoDataFrame(
    pd.read_pickle(data_path),
    geometry="geometry"
)

In [5]:
# select Feature and label
X = np.array([np.array(row['combined_channels']) for _, row in df.iterrows()])
label = np.array(df['label'])

In [6]:
# select only the unique features
# unique_arrays, unique_indices = np.unique(X, axis=-1, return_index=True)
# feature=unique_arrays

# same thing but without waiting
unique_indices = [12, 10, 11, 20, 22, 23, 19,  4, 27, 24,  0,  9, 21,  3,  5, 15, 18, 6]
feature = X[:, :, :, unique_indices]

In [7]:
# splitting
X_train, X_test, y_train, y_test = train_test_split(feature, label, test_size=0.2, random_state=42)

## Dynamic learning rate - decreases over time

In [8]:
def make_model_dynamic_lr(scale=1.0):
    # Define the input shape
    input_shape = (50, 50, 18)

    # Define the input layer
    input_layer = Input(shape=input_shape)

    # Convolutional layers
    # add 1 neuron to have at least 1 neuron with small scale
    conv1 = Conv2D(int(32 * scale) + 1, kernel_size=(3, 3), activation='relu')(input_layer)
    conv2 = Conv2D(int(64 * scale) + 1, kernel_size=(3, 3), activation='relu')(conv1)

    # Flatten the output from convolutional layers
    flattened_input = Flatten()(conv2)

    # Dense layers
    dense1 = Dense(int(512 * scale), activation='relu')(flattened_input)

    # Output layer for binary classification with sigmoid activation
    output_layer = Dense(units=1, activation='sigmoid')(dense1)

    # Create the model
    model = keras.Model(inputs=input_layer, outputs=output_layer)

    learning_rate_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=0.001,
        decay_steps=100,
        decay_rate=0.94,
    )

    # Compile the model for binary classification
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate_schedule), loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [9]:
model_dynamic = make_model_dynamic_lr(scale=0.3)
model_dynamic.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

2024-01-11 16:23:27.726933: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-01-11 16:23:27.726971: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/50


2024-01-11 16:23:32.311283: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-01-11 16:23:32.384952: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_10.


 50/146 [=========>....................] - ETA: 43s - loss: 418.5121 - accuracy: 0.5863